In [1]:
# @zhangjoy
# @主题挖掘
import csv
import time
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import tushare as ts
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer # 词频统计
from sklearn.decomposition import LatentDirichletAllocation # LDA
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf统计
from sklearn.cluster import KMeans # k均值聚类

In [2]:
# 评论文本：comment
filepath = "D:\\Program Files\\Git\\git_work\\weibo-text-project\\cjpl_20180615_20180620.csv"   # 评论内容的存储路径
with open(filepath, 'r') as f:
    reader = csv.reader(f)
    comment = [row[7] for row in reader]
comment.pop(0) # '评论内容'
print(len(comment)) # 6710

505


In [3]:
comment[101]

'来源：火山财富 编辑：东方财富网6月15日晚间，方正证券发布了两则公告，一是瑞士信贷对瑞信方正非公开协议单方面增资至控股股东；二是方正证券对全资子公司方正(香港)金融控股有限公司增资约港币6亿元。瑞信方正2017年实现净利润0.26亿元，此次增资后，瑞士信贷将持有瑞信方正51%的股份，正式成为其控股股东。而另一则公告是方正证券对全资子公司方正香港金控增资约港币6亿元，增资后方正香港金控的注册资本为11亿港元，增资金额比原注册资本还多！火山君注意到，这两条公告相映成趣，一条代表国内券商“走出去”，一条则显示了外资金融机构“走进来”。方正证券子公司即将姓瑞士瑞信方正是方正证券和瑞士信贷于2008年10月共同投资设立的外商投资证券公司，自设立至今，方正证券对瑞信方正的持股比例一直是66.70%。瑞信方正2017年实现营业收入2.70亿元，净利润0.26亿元。瑞信方正的经营范围是股票(包括人民币普通股、外资股)和债券(包括政府债券、公司债券)的承销与保荐；证券经纪业务(限前海地区)。从中可以看出，其经营范围相对比较窄。瑞信方正未经审计的2018年一季度财务数据显示，实现营业收入0.40亿元，净利润-0.18亿元。今年4月28日，证监会公布并实施了《外商投资证券公司管理办法》，规定境外股东持有证券公司的股权比例放宽至51%。瑞士信贷马上行动，拟以直接增资的方式将持股比例提高至51%，对瑞信方正实现控股。火山君注意到，在今年4月底证监会规定外商可投资证券公司的比例提高至51%之后，5月已相继有瑞士银行申请对瑞银证券的持股比例增加至51%、野村证券申请设立控股证券公司、摩根大通申请设立拟持股51%的外商投资证券公司。6月瑞士信贷拟通过增资实现对瑞信方正的控股。此外，火山君注意到，瑞士信贷将控股瑞银方正有助于方正证券解决同业竞争的问题。除了瑞信方正外，方正证券还持有民族证券100%的股份，瑞士信贷的增资完成后，方正证券将不再控股瑞信方正，也就解决了瑞信方正与民族证券之间的同业竞争问题。方正证券大幅增资境外子公司今年以来，多家券商在境外业务上都有不小的动作，方正证券也不例外。方正证券公告称，为实现公司国际化战略，推动方正香港金控业务发展，公司决定对方正香港金控进行增值，增资金额约为港币6亿元。本次增资完成后，方正香港金控注册资本从目前的5亿港元增加至约11亿港元。其实，方正证券去年

In [4]:
# comment可视化分析
l_lst = [len(c) for c in comment]

In [5]:
max(l_lst)

17748

In [6]:
min(l_lst)

48

In [7]:
sum(l_lst) / len(l_lst)

1875.2554455445545

In [44]:
t_lst = [466,511,348,195,689,584,451,355,315,128,123,195,202,178,52,50,51,32,48,43,56,303,302,131,67,272,299,263]

In [45]:
sum(t_lst)

6709

In [5]:
# 得到所有股票的代码和中文名字
stock_name = ts.get_stock_basics()['name'] # 股票名称
stock_code = stock_name.index.tolist() # 股票代码
# 获得所有的概念列表、行业列表、地域列表、公司名称等
concept = ts.get_concept_classified()['c_name'].drop_duplicates().values.tolist() # 概念列表 163
industry = ts.get_industry_classified('sw')['c_name'].drop_duplicates().values.tolist() # 申万行业列表 129 > 110
place = ts.get_area_classified()['area'].drop_duplicates().values.tolist() # 获取地域列表 32 省份

broker = ts.broker_tops()["broker"] # 获取营业部信息：1062
pattern = re.compile(u'.*有限.*?公司') # 最小匹配[*?]到'有限 公司'
company = [" ".join(pattern.findall(s)) for s in broker] # list
company = pd.Series(company).drop_duplicates().values.tolist() # 证券公司：94

[Getting data:]###########################

In [6]:
# 将上述信息作为新词录入词典
for d in stock_name:
    jieba.add_word(d)
for d in stock_code:
    jieba.add_word(d)
for d in concept:
    jieba.add_word(d)
for d in industry:
    jieba.add_word(d)
for d in place:
    jieba.add_word(d)    
for d in company:
    jieba.add_word(d)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/xv/52pzd4c57hb49kf41gxp713w0000gn/T/jieba.cache
Loading model cost 0.927 seconds.
Prefix dict has been built succesfully.


In [8]:
# 启用自定义词典
jieba.load_userdict("D:\\Program Files\\Git\\git_work\\weibo-text-project\\userdict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wu\AppData\Local\Temp\jieba.cache
Loading model cost 0.891 seconds.
Prefix dict has been built succesfully.


In [9]:
# 停用词列表
stop =[line.strip() for line in open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\mystopwords.txt', encoding='utf-8').readlines()]

In [10]:
# 匹配中文
pattern = re.compile(u'[\u4e00-\u9fa5]{2,20}')

In [11]:
text = [] # 清洗后文本
t1 = time.time()
for sentence in comment:
    chinese = pattern.findall(sentence) # 匹配中文：先分句
    seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
    c = []
    for ss in seg:
        for s in ss:
            if s not in stop: # 过滤停用词
                c.append(s)
    text.append(" ".join(c))
t2 = time.time()
print("preprocessing time using %.5f s" %(t2-t1)) # 预处理时间 111.45185 s

preprocessing time using 29.01424 s


In [12]:
text[101]

'火山 财富 晚间 方正证券 两则 公告 瑞士 信贷 对瑞信 方正 非 公开 协议 单方面 增 资至 控股股东 方正证券 全资 子公司 方正 香港 金融 控股有限公司 增资 约 港币 瑞信 方正 净利润 增资 瑞士 信贷 将持 有瑞信 方正 股份 正式成为 控股股东 另一则 公告 方正证券 全资 子公司 方正 香 港金控 增资 约 港币 增资 后方 正 香港 金控 注册资本 亿港元 增 资金额 比原 注册资本 火山 君 两条 公告 相映成趣 一条 代表 国内券商 走 一条 外资金融机构 走进 方正证券 子公司 姓 瑞士 瑞信方 方正 证 券 瑞士 信贷 共同 投资 设立 外商投资 证券公司 设立 方正证券 对瑞信 方正 持股比例 瑞信 方正 营业 收入 净利润 瑞信 方正 经营范围 股票 人民币 普通股 外资股 债券 政府债券 公司债券 承销 保荐 证券 经纪业务 限 前海 地区 经营范围 相对 较窄 瑞信 方正 未经 审计 财务数据 营业 收入 净利润 证监会 公布 外商投资 证券公司 管理办法 规定 境外 股东 持 证券公司 股权 比例 放宽 瑞士 信贷 行动 拟以 直接 增资 方式 持股比例 提高 对瑞信 方正 控股 火山 君 月底 证监会 规定 外商 投资 证券公司 比例 提 高至 相继 瑞士银行 申请 瑞银 证券 持股 例 增加 野村 证券 申请 设立 控股 证券公司 摩根大通 申请 设立 拟 持股 外商投资 证券公司 瑞士 信贷 拟 增资 对瑞信 方正 控股 火山 君 瑞士 信贷 控股 瑞银 方正 有助于 方正证券 解决 同业竞争 瑞信 方正 外 方正证券 还持 民族证券 股份 瑞士 信贷 增资 完成 方正证券 控股 瑞信 方正 解决 瑞信 方正 民族证券 同业 竞 争 方正证券 大幅 增资 境外 子公司 券商 境外 业务 不小 动作 方正证券 例外 方正证券 公告称 国际化 战略 方正 香港 金控 业务发展 正 香港 金控 增值 增 资金额 约 港币 增资 完成 方正 香港 金控 注册资本 亿港元 增加 至约 亿港元 方正证券 完成 正 香港 金控 亿港元 增资 翻倍 追加 火山 君 方正证券 使用 港币 境外 子公司 业务 重点 开拓 做 做 强 经纪 资管 业务 新 设 业务 协同 方式 开展 投行 固守 重点 核心 业务 打造 金融 科技 平台 

In [13]:
# 使第6708条评论完美分词
jieba.load_userdict("D:\\Program Files\\Git\\git_work\\weibo-text-project\\userdict.txt")
stop =[line.strip() for line in open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\mystopwords.txt', encoding='utf-8').readlines()]
text_101 = "" # 清洗后文本
comment_101 = comment[101]
chinese = pattern.findall(comment_101) # 匹配中文：先分句
seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
c = []
for ss in seg:
    for s in ss:
        if s not in stop: # 过滤停用词
            c.append(s)
text_101 = " ".join(c)
print(text_101)

火山 财富 晚间 方正证券 两则 公告 瑞士 信贷 对瑞信 方正 非 公开 协议 单方面 增 资至 控股股东 方正证券 全资 子公司 方正 香港 金融 控股有限公司 增资 约 港币 瑞信 方正 净利润 增资 瑞士 信贷 将持 有瑞信 方正 股份 正式成为 控股股东 另一则 公告 方正证券 全资 子公司 方正 香 港金控 增资 约 港币 增资 后方 正 香港 金控 注册资本 亿港元 增 资金额 比原 注册资本 火山 君 两条 公告 相映成趣 一条 代表 国内券商 走 一条 外资金融机构 走进 方正证券 子公司 姓 瑞士 瑞信方 方正 证 券 瑞士 信贷 共同 投资 设立 外商投资 证券公司 设立 方正证券 对瑞信 方正 持股比例 瑞信 方正 营业 收入 净利润 瑞信 方正 经营范围 股票 人民币 普通股 外资股 债券 政府债券 公司债券 承销 保荐 证券 经纪业务 限 前海 地区 经营范围 相对 较窄 瑞信 方正 未经 审计 财务数据 营业 收入 净利润 证监会 公布 外商投资 证券公司 管理办法 规定 境外 股东 持 证券公司 股权 比例 放宽 瑞士 信贷 行动 拟以 直接 增资 方式 持股比例 提高 对瑞信 方正 控股 火山 君 月底 证监会 规定 外商 投资 证券公司 比例 提 高至 相继 瑞士银行 申请 瑞银 证券 持股 例 增加 野村 证券 申请 设立 控股 证券公司 摩根大通 申请 设立 拟 持股 外商投资 证券公司 瑞士 信贷 拟 增资 对瑞信 方正 控股 火山 君 瑞士 信贷 控股 瑞银 方正 有助于 方正证券 解决 同业竞争 瑞信 方正 外 方正证券 还持 民族证券 股份 瑞士 信贷 增资 完成 方正证券 控股 瑞信 方正 解决 瑞信 方正 民族证券 同业 竞 争 方正证券 大幅 增资 境外 子公司 券商 境外 业务 不小 动作 方正证券 例外 方正证券 公告称 国际化 战略 方正 香港 金控 业务发展 正 香港 金控 增值 增 资金额 约 港币 增资 完成 方正 香港 金控 注册资本 亿港元 增加 至约 亿港元 方正证券 完成 正 香港 金控 亿港元 增资 翻倍 追加 火山 君 方正证券 使用 港币 境外 子公司 业务 重点 开拓 做 做 强 经纪 资管 业务 新 设 业务 协同 方式 开展 投行 固守 重点 核心 业务 打造 金融 科技 平台 开

In [14]:
corpus = text # 清洗后文本作为语料

In [15]:
# LDA
# 特征处理: 词频做文档特征
# 构建 Document-word matrix
count_vectorizer = CountVectorizer(stop_words=stop) # 停用词, 特征选择：min_df = 6
count = count_vectorizer.fit_transform(corpus) # 文档-词频矩阵：<class 'scipy.sparse.csr.csr_matrix'>
word_names = count_vectorizer.get_feature_names() # wordlist 长度：84888

In [16]:
countpie_vectorizer = CountVectorizer(min_df=50,stop_words=stop) # 过滤词出现的文档数 < 50 的词 ：特征提取
countpie = countpie_vectorizer.fit_transform(corpus) # 文档-词频矩阵：<class 'scipy.sparse.csr.csr_matrix'>
print(countpie.shape)

(505, 314)


In [17]:
c_lst = [count.sum(axis=0)[0,i] for i in range(len(word_names))] # 每个词的词频列表 耗时：5m

In [18]:
less_6 = sum([Counter(c_lst)[i] for i in range(6)]) # 词频 < 6 的个数

In [19]:
# 特征选择：基于文档频率，返回文档频率排名前 K 的词语
def word_count_select(df,K):
    df_mean = df.mean() # 求每列的均值/和等
    # 基于 文档频率排序 选择特征，一定返回K个词语
    WordTF_series = df_mean.sort_values(ascending=False) # 降序排序
    WordTF_topK = WordTF_series[:K] # 选择前 K 的特征
    word_feature = WordTF_topK.index # word_name [pandas.core.indexes.base.Index]
    feature_list = list(WordTF_topK.index) # list(word_name)
    return word_feature,feature_list
# 词频小于 6 的个数统计
def count_less_n(lst,N):
    lessN = sum([Counter(lst)[i] for i in range(N)]) # 词频 < N 的个数
    return lessN

In [20]:
# 把文档-词语矩阵转为数据框格式
count_ndarray = count.toarray()
count_df = DataFrame(count_ndarray, columns=word_names)
k = len(word_names) - count_less_n(c_lst,6) # 词频 > 6 的个数

wordstay_names,wordstay_names_list = word_count_select(count_df,k)
count_df2 = count_df[wordstay_names]
countstay = count_df2.as_matrix(columns=wordstay_names_list) # matrix

In [21]:
# count # 6709x84884 sparse matrix numpy.int64
# 去掉低频词：在主题模型中，数据预处理需要去掉低频词，大多数阈值设定为5或6,此处设定为6
# matrix是numpy中的特殊类型，作为array的子类出现，所以继承了array的所有特性并且有自己的特殊的地方，专门用来处理线性代数操作

In [22]:
# 词频数分布情况
f = open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\count_1.txt','w')
f.write(str([Counter(c_lst).most_common(100)[i][0] for i in range(100)]))
f.close()
f = open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\count_2.txt','w')
f.write(str([Counter(c_lst).most_common(100)[i][1] for i in range(100)]))
f.close()

In [23]:
# 主题挖掘建模：LDA算法
# 训练模型
k = 50 # 选择主题数
lda_50 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_50.fit(countstay)
# 6709文档，单词数：25434，分50个主题 耗时：< 10m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=50, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [24]:
# 训练模型
k = 30 # 选择主题数
lda_30 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_30.fit(countstay)
# 6709文档，单词数：25434，分30个主题 耗时：12:54-13:01 7m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=30, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [25]:
# 训练模型
k = 10 # 选择主题数
lda_10 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_10.fit(countstay)
# 6709文档，单词数：25434，分10个主题 耗时：13:02-13:08 6m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [26]:
# 结果展示
# Topic Words 结果
# topicword_matrix = lda.components_ # numpy.ndarray
# print(topicword_matrix[0,1]) # topicword_matrix[i,j]: word j was assigned to topic i
# print(topicword_matrix.shape)
# topicword_matrix / topicword_matrix.sum(axis=1)[:, np.newaxis] # normalization

# Topic（Top）Words结果
def print_top_words(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_ # Topic Words 分布结果 numpy.ndarray
    for topic_idx, topic in enumerate(topicword_matrix):
        print("Topic # %d: " % topic_idx)
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2)) + '| ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

def print_top_words_name(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_
    for topic_idx, topic in enumerate(topicword_matrix):
#         print("Topic # %d:" % topic_idx)
        print("Topic %d:" % topic_idx, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [27]:
# Doc Topics 结果
# 通过fit_transform函数,可以得到文档主题分布,Doc_Topic distribution matrix
# doctopic_matrix = lda.fit_transform(dw_countvector) # numpy.ndarray
# print(doctopic_matrix)
# print(doctopic_matrix.shape)

# Doc（Top）Topic结果
def print_top_topics(model, feature_names, n_top_topics):
#     fo = open("/Users/zhangjoy/Documents/vscode/py_project/lda/data/doc_top_topics.txt", "rw+")
    print("# 打印每个文档下权重较高的主题")
    doctopic_matrix = model.fit_transform(countstay)
    for doc_idx, doc in enumerate(doctopic_matrix):
        print("Doc # %d:" % doc_idx)
        print(''.join([str(feature_names[i]) + ' ' + str(round(doc[i], 2)) + '| ' for i in doc.argsort()[:-n_top_topics - 1:-1]]))

In [28]:
print_top_words_name(lda_50, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
Topic 0: 西安 供应 住房 外商投资 部门 投资 经济 美国 国家 开发区
Topic 1: 登记 不动产 账户 查询 证券 平台 统一 中国 房价 房地产
Topic 2: 银行 分红 记者 数据 方案 年度 贷款 上浮 现金 同比
Topic 3: 行业 板块 增长 持续 消费 预计 提升 业绩 预期 龙头
Topic 4: 独角兽 股市 股票 投资者 概念 新经济 指数 投资 涨停 资金
Topic 5: 券商 苹果 中国 业务 记者 证券 财富 端午节 业绩 净利润
Topic 6: 好未来 报告 浑水 富士康 做空 股价 业务 交易 澳洋科技 中国
Topic 7: 基金 战略 独角兽 配售 国家队 产品 补贴 注射剂 销量 基金公司
Topic 8: 唐小僧 员工持股计划 项目 投资 商行 投资者 资邦 平台 员工 宋城演艺
Topic 9: 创新 存托 凭证 试点 投资者 红筹 投票权 上市 境内 证监会
Topic 10: 资金 外资 美国 投资 预期 指数 美股 估值 限制 调整
Topic 11: 增速 安信信托 下滑 投资 购买 数据 工业 开户 回升 回落
Topic 12: 管网 天然气 油气 石油 国家 开放 管道 第三方 资产 发改委
Topic 13: 技术 产业 应急 发展 中国 产品 服务 领域 创新 平台
Topic 14: 存托 凭证 战略 业务 中国 中心 结算 舆情 研究 投资者
Topic 15: 上市公司 公司治理 准则 改革 发展 资本市场 证监会 治理 制度 机构
Topic 16: 银隆 珠海 上市公司 记者 颜静刚 格力 上海 格力电器 收购 投资
Topic 17: 斯巴鲁 苏宁 中国 激光 零售 销售 电商 品牌 增长 用户
Topic 18: 审核 业务 收入 记者 湖南 发审委 申报 支付 保荐 机构
Topic 19: 多点 数码 物美 东蓝 飞利信 世界杯 数字化 梅安森 业绩 预测
Topic 20: 新能源 汽车 北汽 结构 国家 中国 投资 建设 人民币 融资
Topic 21: 河北省 落实 百分百 六个 组织 排放 建筑工地 北京市 工业 粉尘
Topic 22: 存款 商行 银行 资金 贴息 支行 存单 储户 吸收 伪造
Topic 23: 增长 

In [29]:
print_top_words_name(lda_30, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
Topic 0: 净流入 主力资金 营业部 买入 净流出 交易日 集体 金额 数据 行业
Topic 1: 信息 交易 虚假 案件 行为 办案 违法行为 调查 涉嫌 传播
Topic 2: 增长 行业 同比 调研 预计 持续 机构 需求 提升 业绩
Topic 3: 银隆 珠海 格力 格力电器 记者 员工 苹果 长园集团 汽车 丰田
Topic 4: 加息 美国 美联储 美元 央行 利率 预期 经济 指数 土地
Topic 5: 小米 金融 京东 业务 互联网 证监会 发行 生态 估值 服务
Topic 6: 增速 上市公司 准则 公司治理 治理 销售 修订 数码 投资 下滑
Topic 7: 账户 证券 指数 新经济 管理 股票 措施 中国 结算 限制
Topic 8: 零售 电商 京东 苏宁 多点 管网 平台 用户 消费者 小龙虾
Topic 9: 存托 凭证 创新 投资者 试点 红筹 投票权 股票 境外 规定
Topic 10: 公告 项目 股份 投资 股权 上市公司 记者 控股股东 资产 万股
Topic 11: 网络 通信 独立 组网 商用 行业 运营商 充电桩 移动 建设
Topic 12: 美国 关税 中国 美方 进口 加征 商品 中方 贸易 措施
Topic 13: 新能源 汽车 技术 产品 日本 北汽 半导体 领域 中国 制造
Topic 14: 中国 经济 股市 资金 货币 国内 发展 融资 央行 增长
Topic 15: 质押 比例 股票 平仓 股东 资金 股权 控股股东 增持 股份
Topic 16: 估值 预期 投资 消费 下半年 底部 股市 风险 历史 机会
Topic 17: 山东 商行 上市公司 发展 产业 建设 项目 经济 煤炭 江苏
Topic 18: 应急 生产 食品安全 食品 产业 中国 装备 重工 规范 检查
Topic 19: 足球 融资 上市 发展 定价 患者 创业 中国 药明康德 国内
Topic 20: 基金 银行 分红 战略 中粮 配售 独角兽 资本 国家队 规模
Topic 21: 下跌 板块 指数 投资者 股价 震荡 新低 好未来 跌幅 沪指
Topic 22: 机构 估值 板块 行业 资金 策略 股票 评级 上市公司 买入
Topic 23: 国企 国企改革 打印 改革 唐小僧

In [30]:
print_top_words_name(lda_10, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
Topic 0: 世界杯 美国 中国 全球 美元 经济 足球 俄罗斯 赞助商 日本
Topic 1: 质押 加息 资金 央行 美联储 股票 平仓 比例 利率 风险
Topic 2: 行业 增长 增速 同比 券商 投资 板块 消费 持续 调研
Topic 3: 上市公司 银行 股权 记者 股份 股东 分红 净利润 资金 商行
Topic 4: 城市 项目 公告 摇号 销售 房价 土地 价格 记者 调控
Topic 5: 小米 基金 证监会 发行 登记 不动产 投资者 试点 创新 审核
Topic 6: 改革 上市公司 落实 河北省 发展 国企 投资 国企改革 工业 国家
Topic 7: 估值 股市 独角兽 股票 指数 买入 投资者 股价 净流入 投资
Topic 8: 存托 凭证 创新 投资者 红筹 投票权 规定 试点 交易 境外
Topic 9: 中国 技术 新能源 汽车 京东 金融 发展 产品 业务 产业


In [31]:
# 训练模型
k = 20 # 选择主题数
lda_20 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_20.fit(countstay)
# 6709文档，单词数：25434，分20个主题 耗时：17:28-17:35 7m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [32]:
print_top_words_name(lda_20, wordstay_names_list, 30)

# 打印每个主题下权重较高的词
Topic 0: 增速 技术 半导体 激光 好未来 产业 教育 项目 手机 投资 中国 打印 领域 国内 做空 业务 同比 芯片 浑水 报告 华谊兄弟 增长 全球 产品 工业 落户 发展 电子 制造 未来
Topic 1: 质押 不动产 登记 股权 上市公司 股份 平仓 控股股东 比例 股东 记者 股价 持股 股票 查询 风险 颜静刚 上海 总股本 研究 市值 亿股 信息 交易 资金 中药 资产 统一 战略 中心
Topic 2: 基金 公告 项目 股份 投资 万股 收入 股权 子公司 控股股东 方正 总股本 增持 募集 增资 净利润 减持 产品 业绩 重大 员工持股计划 投资者 股票 计划 战略 认购 方正证券 数码 业务 完成
Topic 3: 金融 京东 服务 业务 平台 合作 科技 银行 领域 支付 消费 产品 金融机构 提供 特斯拉 估值 数据 用户 上线 收入 大数据 蚂蚁金服 牌照 供应链 国内 未来 财富 模式 优势 金融服务
Topic 4: 应急 摇号 记者 项目 房源 楼盘 分级 中粮 成都 购房者 南京 中签率 产业 西安 装备 中国 重工 二手房 平方米 资本 开盘 购房 河西 客户 楼市 套房 销售 均价 价格 区域
Topic 5: 估值 板块 投资 股市 资金 预期 消费 行业 投资者 策略 下半年 下跌 机会 股票 风险 整体 因素 底部 杠杆 周期 行情 历史 短期 基本面 持续 业绩 指数 长期 医药 创新
Topic 6: 资金 投资 基金 苏宁 交易 股票 外资 限制 账户 球票 丰田 私募 特大 凡得 净流入 北向 机构 苏宁易购 汇出 资产 信息 督察 买入 入市 境外 管理 取消 刘晓东 幸福 西蒙斯
Topic 7: 煤炭 沪指 闪崩 跌停 震荡 板块 反弹 新低 指数 山东 计划 概念 大盘 盘面 市值 加息 机会 美联储 护盘 机构 早盘 跌破 信托 投资者 预测 银行 五大 午后 钢铁 阶段
Topic 8: 银行 股价 净流入 买入 数据 主力资金 营业部 增持 金额 涨幅 交易日 股东 独角兽 富士康 净流出 行业 共有 国家队 安信信托 上涨 证券 下跌 配售 战略 龙虎榜 工业富联 股市 股票 总市值 最大
Topic 9: 存托 凭证 创新 投资者 红筹 投票权 试点 

In [37]:
# 训练模型
k = 30 # 选择主题数
lda = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda.fit(countstay)
# 6709文档，单词数：84884，分20个主题 耗时：2-3m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=30, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [39]:
print_top_words_name(lda, wordstay_names, 15)

# 打印每个主题下权重较高的词
Topic 0: 经济 央行 加息 中国 美联储 利率 美国 增长 货币 杠杆 未来 货币政策 国企改革 违约 流动性
Topic 1: 世界杯 中国 足球 俄罗斯 赞助商 赞助 体育 全球 球迷 国际足联 品牌 英利 球票 万达 巴西
Topic 2: 银行 分红 净流入 主力资金 股价 案件 资金 金额 净流出 数据 信息 共有 特大 涨幅 办案
Topic 3: 增速 同比 增长 销售 土地 数据 工业 城市 需求 全国 行业 国企 结构 价格 下滑
Topic 4: 投资 中心 唐小僧 研究 中药 战略 舆情 记者 丰田 注射剂 平台 资邦 中国 合作 玛丽
Topic 5: 独角兽 估值 美国 股票 指数 股市 策略 调整 美股 经济 成长 预期 收益率 国家队 增长
Topic 6: 指数 下跌 估值 投资者 股票 新低 风险 预期 沪指 跌停 股市 底部 板块 白酒 大盘
Topic 7: 不动产 登记 城市 房价 查询 房地产 商品住宅 调控 新建 环比 全国 住宅 销售价格 统一 上涨
Topic 8: 小米 证监会 发行 试点 审核 创新 定价 投资者 估值 监管 询价 上市 机构 申请 反馈
Topic 9: 发展 改革 建设 国家 我国 进口 中国 环保 经营 服务 投资 领域 提升 部门 完善
Topic 10: 公告 项目 股份 上市公司 股权 控股股东 子公司 万股 总股本 方正 增持 增资 收入 资产 投资
Topic 11: 摇号 西安 记者 房源 项目 城市 楼盘 成都 购房者 南京 套房 买房 中签率 二手房 住房
Topic 12: 资金 增持 中粮 外资 买入 净流入 资本 机构 改革 投资 限制 中国 融资 北向 采购
Topic 13: 金融 京东 新能源 汽车 品牌 北汽 产品 业务 苏宁 增长 服务 销量 领域 中国 技术
Topic 14: 存托 凭证 创新 红筹 投资者 投票权 试点 股票 规定 交易 境外 境内 上市 监管 实施办法
Topic 15: 技术 产业 中国 全球 发展 日本 半导体 行业 通信 网络 独立 国家 领域 经济 产品
Topic 16: 美元 黄金 商行 期货 主席 美国 收跌 欧洲央行 金价 员工持股计划 收涨 计划 收盘 上海 员工
Topic 17

In [40]:
# Doc Topics 结果
# 通过fit_transform函数,可以得到文档主题分布,Doc_Topic distribution matrix
doctopic_matrix = lda.fit_transform(count) # numpy.ndarray
print(doctopic_matrix)
print(doctopic_matrix.shape)

[[1.73611111e-04 1.73611111e-04 1.73611111e-04 ... 1.73611111e-04
  8.18706560e-01 1.73611111e-04]
 [1.85185185e-03 1.85185185e-03 1.85185185e-03 ... 1.85185185e-03
  5.03672518e-01 1.85185185e-03]
 [1.10011001e-04 1.10011001e-04 1.10011001e-04 ... 1.10011001e-04
  1.10011001e-04 1.10011001e-04]
 ...
 [1.23456790e-04 1.23456790e-04 1.23456790e-04 ... 1.23456790e-04
  1.23456790e-04 1.23456790e-04]
 [1.31233596e-04 1.31233596e-04 1.31233596e-04 ... 1.31233596e-04
  1.31233596e-04 1.31233596e-04]
 [8.14995925e-05 8.14995925e-05 8.14995925e-05 ... 8.14995925e-05
  9.97636512e-01 8.14995925e-05]]
(505, 30)


In [41]:
def top_topics(model,topN):
    doctopic_matrix = model.fit_transform(countstay)
    doctopic = doctopic_matrix.argmax(axis=1).tolist() # 取最大值时的索引
    print("# 打印前 %d 热门主题的序号" % topN)
    print([Counter(doctopic).most_common(topN)[i][0] for i in range(topN)]) # 找出列表元素最多的N个,结果是list类型

In [45]:
top_topics(lda,30)
# 6709个文档，lda_50，前10个主题，耗时： 8 m 

# 打印前 30 热门主题的序号
[3, 21, 17, 15, 22, 23, 0, 10, 25, 13, 29, 14, 5, 11, 27, 24, 28, 18, 2, 4, 6, 20, 26, 9, 12, 16, 1, 7, 19, 8]


In [44]:
# # 效果等同上述函数
# doctopic_matrix = lda.fit_transform(countstay)
# doctopic = doctopic_matrix.argmax(axis=1).tolist() # 取最大值时的索引
# print("# 打印前 %d 热门主题的序号" % topN)
# c = Counter(doctopic)
# c.most_common(10) # 找出列表元素最多的N个,结果是list类型

In [76]:
# 主题挖掘建模：文本聚类算法
# 特征处理: TF-IDF做文档特征
tfidf_vectorizer = TfidfVectorizer(stop_words=stop,max_df=0.9,min_df=10) # 特征提取
tfidf = tfidf_vectorizer.fit_transform(corpus) # matrix 
word_names = tfidf_vectorizer.get_feature_names() # list
# tfidf矩阵:元素a[i][j]表示j词在i文档中的tf-idf权重
# weight = tfidf.toarray()  # matrix -> ndarray
print(len(word_names))

14839


In [75]:
# 定义余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf)

In [78]:
# keans
# 训练模型
k = 20
km_20 = KMeans(n_clusters=k)
%time km_20.fit(tfidf)
clusters = km_20.labels_.tolist() # 每个文档所属类别

CPU times: user 3min 36s, sys: 2.12 s, total: 3min 38s
Wall time: 3min 41s


In [109]:
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km_20.cluster_centers_.argsort()[:, ::-1] 

for i in range(k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :15]: # 每个聚类选 15 个词
        print(' %s' % word_names[ind], end=' ')
    print() # 空行

Top terms per cluster:

Cluster 0: 人民币  美元  汇率  升值  中间价  在岸  美元汇率  货币  基点  中国  贬值  指数  官方  交易日  收盘价 
Cluster 1: 增持  股份  公告  股东  回购  万股  控股  股本  计划  公司股票  减持  上市公司  晚间  员工  低于 
Cluster 2: 卖出  买入  机构  金额  个股  龙虎榜  营业部  融资  当天  资金  数据  交易  万股  席位  证券 
Cluster 3: 美股  股市  下跌  指数  美国  暴跌  经济  估值  大跌  调整  跌幅  全球  影响  风险  预期 
Cluster 4: 企业  发展  中国  经济  改革  建设  产业  金融  创新  项目  资本  国家  投资  行业  服务 
Cluster 5: 板块  反弹  行情  个股  创业板  指数  估值  业绩  大盘  资金  调整  下跌  春节  上涨  震荡 
Cluster 6: 基金  比特币  货币  开户  港股  投资  交易  购买  股票  虚拟  乐视网  投资者  持有  资金  指数 
Cluster 7: 分红  现金  上市公司  股息  说明会  上交所  投资者  回报  监管  铁公鸡  具备  东安动力  分配利润  利润分配  联美控股 
Cluster 8: 中国  股份  增长  投资  企业  集团  业务  上市  产品  公告  发行  项目  行业  销售  股东 
Cluster 9: 主力  净流入  资金  流出  个股  乖离  三日  涨跌幅  证券  净流量  最多  排名  金额  半年线  概念 
Cluster 10: 业绩  净利润  增长  同比  预计  预告  行业  年报  板块  实现  上市公司  增速  个股  收入  披露 
Cluster 11: 美联储  美国  加息  预期  收益率  经济  指数  通胀  前值  美元  股市  鲍威尔  美股  利率  国债 
Cluster 12: 万达  腾讯  阿里  零售  饿了么  新零售  步步高  京东  线下  王健林  商业  阿里巴巴  合作  集团  万达电影 
Cl

In [113]:
km_20.labels_ # numpy.ndarray

array([17,  6, 10, ...,  8, 17, 11], dtype=int32)

In [105]:
[Counter(km_20.labels_.tolist()).most_common(20)[i][0] for i in range(20)]

[8, 4, 5, 10, 17, 3, 16, 6, 11, 14, 19, 15, 2, 9, 1, 12, 18, 0, 7, 13]

In [106]:
Counter(km_20.labels_.tolist()).most_common(20)

[(8, 2564),
 (4, 808),
 (5, 518),
 (10, 454),
 (17, 268),
 (3, 237),
 (16, 211),
 (6, 207),
 (11, 195),
 (14, 176),
 (19, 173),
 (15, 151),
 (2, 141),
 (9, 141),
 (1, 128),
 (12, 85),
 (18, 75),
 (0, 62),
 (7, 59),
 (13, 56)]